## Automatic Vectorization in JAX

- Original: [JAX Tutorial: Automatic Vectorization in JAX](https://jax.readthedocs.io/en/latest/jax-101/03-vectorization.html)
- Korean: [JAX 튜토리얼: JAX의 자동 벡터화](https://jax-kr.readthedocs.io/ko/latest/JAX101/%ED%98%95%EC%84%AD%EC%B4%88%EB%B2%8C_Jitting_functions_in_JAX.html)

### 수동 벡터화

In [1]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
w = jnp.array([2., 3., 4.])

def convolve(x, w):
    output = []
    for i in range(1, len(x)-1):
        output.append(jnp.dot(x[i-1:i+2], w))
    return jnp.array(output)

display(convolve(x, w))

CUDA backend failed to initialize: Found cuBLAS version 120100, but JAX was built against version 120304, which is newer. The copy of cuBLAS that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Array([11., 20., 29.], dtype=float32)

- 위 함수를 각각 벡터와 가중치의 batch인 `x`와 `w`에 적용하고 싶다고 가정

- 가장 단순한 파이썬 방법은 배치를 루프로 반복하는 것

In [2]:
xs = jnp.stack([x, x])
ws = jnp.stack([w, w])

In [7]:
print(xs)
print(ws)

[[0 1 2 3 4]
 [0 1 2 3 4]]
[[2. 3. 4.]
 [2. 3. 4.]]


In [3]:
def manually_batched_convolve(xs, ws):
    output = []
    for i in range(xs.shape[0]):
        output.append(convolve(xs[i], ws[i]))
    return jnp.stack(output)

display(manually_batched_convolve(xs, ws))

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

- 위 연산을 효율적으로 일괄 처리하려면, 일반적으로 함수를 벡터화된 형태로 수행되도록 재작성해야 함

    - 구현하기 어려운 방식은 아니지만, 함수가 index, axis 및 입력의 다른 부분들을 처리하는 방식을 변경해야 함

In [4]:
def manually_vectorized_convolve(xs, wx):
    output = []
    for i in range(1, x.shape[-1] - 1):
        output.append(jnp.sum(xs[:, i-1:i+2] * ws, axis=1))
    return jnp.stack(output, axis=1)

manually_vectorized_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

### 자동 벡터화

- JAX에서는 `jax.vmap` 변환을 통해 벡터화 함수 구현을 자동으로 생성할 수 있도록 함

In [5]:
auto_batch_convolve = jax.vmap(convolve)
display(auto_batch_convolve(xs, ws))

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

- `jax.jit`과 유사하게 함수를 tracing하고, 각 입력 시작 부분에 배치의 축을 자동으로 추가하는 방식으로 수행

- 만약 배치 차원이 첫 번째 차원이 아닌 경우, `in_axes` 및 `out_axes` argument를 사용해 입력과 출력에서 배치 차원의 위치를 지정할 수 있음

    - 배치 축이 모든 입출력에 대해 동일한 경우 `int`가 될 수 있으며, 그렇지 않은 경우 `list`도 가능함

In [8]:
auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=1)

xst = jnp.transpose(xs)
wst = jnp.transpose(ws)

display(auto_batch_convolve_v2(xst, wst))

Array([[11., 11.],
       [20., 20.],
       [29., 29.]], dtype=float32)

- `jax.vmap`은 argument 중 하나만 일괄 처리되는 경우도 지원함

    - 예를 들어 벡터 `x`를 일괄처리하여 가중치 `w`의 단일 집합으로 convolution하려는 경우, `in_axes` argument를 `None`으로 설정할 수 있음

In [9]:
batch_convolve_v3 = jax.vmap(convolve, in_axes=[0, None])
display(batch_convolve_v3(xs, w))

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

### Transformation 결합

- 모든 JAX 변환과 마찬가지로, `jax.jit` 및 `jax.vmap`은 composable하게 설계되었음

- 즉, `vmap`된 함수를 JIT으로 wrapping하거나, JITted 함수를 `vmap`으로 wrapping해도 올바르게 동작함

In [10]:
jitted_batch_convolve = jax.jit(auto_batch_convolve)
display(jitted_batch_convolve(xs, ws))

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)